In [52]:
import pandas as pd
import numpy as np
from datetime import datetime

#Statistical LTV
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data, summary_data_from_transaction_data

#ML Approach to LTV
import lightgbm as lgb

#Evaluation
from sklearn.metrics import mean_absolute_error

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#### **Data Pre-Processing**

In [11]:
retaildf = pd.read_csv('data/CLV_online_retail.csv', engine='python')

In [12]:
retaildf

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/11 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,12680.0,France


In [15]:
retaildf['InvoiceDate'] = pd.to_datetime(retaildf.InvoiceDate, format = '%m/%d/%y %H:%M')

In [16]:
#separate out the various parts of the date
retaildf['date'] = pd.to_datetime(retaildf.InvoiceDate.dt.date)
retaildf['time'] = retaildf.InvoiceDate.dt.time
retaildf['hour'] = retaildf['time'].apply(lambda x: x.hour)
retaildf['weekend'] = retaildf['date'].apply(lambda x: x.weekday() in [5, 6])
retaildf['dayofweek'] = retaildf['date'].apply(lambda x: x.dayofweek)

In [18]:
retaildf.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,time,hour,weekend,dayofweek
138900,548219,22719,GUMBALL MONOCHROME COAT RACK,2,2011-03-30 09:46:00,2.46,NaN,United Kingdom,2011-03-30,09:46:00,9,False,2
331673,566025,22671,FRENCH LAUNDRY SIGN BLUE METAL,2,2011-09-08 12:55:00,1.65,17634.0,United Kingdom,2011-09-08,12:55:00,12,False,3
533570,581064,21592,RETROSPOT CIGAR BOX MATCHES,48,2011-12-07 11:16:00,0.39,13882.0,United Kingdom,2011-12-07,11:16:00,11,False,2
337076,566427,85049G,CHOCOLATE BOX RIBBONS,3,2011-09-12 14:24:00,1.25,17433.0,United Kingdom,2011-09-12,14:24:00,14,False,0
159210,550328,22649,STRAWBERRY FAIRY CAKE TEAPOT,2,2011-04-17 13:18:00,4.95,17582.0,United Kingdom,2011-04-17,13:18:00,13,True,6


In [22]:
salesdf = retaildf.groupby('date')['Quantity'].sum()
fig = px.line(salesdf, title='Sales')
fig.update_layout(
    xaxis_title = 'Date',
    yaxis_title = 'Quantity'
)
fig.show()
print(retaildf['date'].max() - retaildf['date'].min())

373 days 00:00:00


##### From this plot, we see that we have about 1 year of data. For our ML approach (Random Forest Regression), the train/test splits will be the following segments:
##### &emsp;1. Training Features Period: 2011-01-01 to 2011-06-11
##### &emsp;2. Training Target Period: 2011-06-12 to 2011-09-09
##### &emsp;3. Testing Features Period: 2011-04-02 to 2011-09-10
##### &emsp;4. Testing Target Period: 2011-09-11 to 2011-12-09

##### The choice of time periods was arbitrary. My goal was to have my target period be roughly half the duration of my training period for the features and target.

In [23]:
#Dataset info
print(f'Total Number of Purchases: {retaildf.shape[0]}')
print(f'Total Number of transactions: {retaildf.InvoiceNo.nunique()}')
print(f'Total Unique Days: {retaildf.date.nunique()}')
print(f'Total Unique Customers: {retaildf.CustomerID.nunique()}')
print(f"We are predicting {(retaildf['date'].max() - datetime(2011, 9, 11)).days} days")

Total Number of Purchases: 541909
Total Number of transactions: 25900
Total Unique Days: 305
Total Unique Customers: 4372
We are predicting 89 days


#### **Baseline model: BG/NBD and Gamma-Gamma models for CLV**
##### Check out the **BGNBD_analysis.ipynb** file in this repository for a more in-depth walkthrough of the implementation of these models for predicting CLV. In this experiment, I am just going to fit and evaluate the models and focus more on the ML technique.

##### **Data Prep**
##### This modeling technique requires data on the transaction level, and the features need to be in RFM format (must have *recency*, *frequency*, *monetary_value*, and *T*(age) columns). For this data, I will calculate revenue as the product of quantity and item price.

In [24]:
#Get revenue column
retaildf['Revenue'] = retaildf['Quantity'] * retaildf['UnitPrice']

#Context data for the revenue (date & customerID)
id_lookup = retaildf[['CustomerID', 'InvoiceNo', 'date']].drop_duplicates()
id_lookup.index = id_lookup['InvoiceNo']
id_lookup = id_lookup.drop('InvoiceNo', axis=1)

transactions_data = pd.DataFrame(retaildf.groupby('InvoiceNo')['Revenue'].sum()).join(id_lookup)

In [25]:
transactions_data.head()

,Revenue,CustomerID,date
InvoiceNo,,,
536365,139.12,17850.0,2010-12-01
536366,22.20,17850.0,2010-12-01
536367,278.73,13047.0,2010-12-01
536368,70.05,13047.0,2010-12-01
536369,17.85,13047.0,2010-12-01


In [27]:
#Spit into train - test
rfm_train_test = calibration_and_holdout_data(transactions_data, 'CustomerID', 'date',
                                        calibration_period_end='2011-09-10',
                                        monetary_value_col = 'Revenue')   

#Selecting only customers with positive value in the calibration period (otherwise Gamma-Gamma model doesn't work)
rfm_train_test = rfm_train_test.loc[rfm_train_test['monetary_value_cal'] > 0, :]

In [29]:
print(rfm_train_test.shape)
rfm_train_test.head()

(1965, 7)


,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
CustomerID,,,,,,,
12347.0,4.0,238.0,277.0,519.7675,2.0,759.570000,90.0
12348.0,2.0,110.0,268.0,297.2200,1.0,310.000000,90.0
12352.0,3.0,34.0,206.0,101.5600,3.0,314.743333,90.0
12356.0,1.0,80.0,235.0,481.4600,1.0,58.350000,90.0
12359.0,3.0,142.0,241.0,970.8100,2.0,1392.800000,90.0


##### **Training**
##### We need to make sure that montary value and frequency are not highly correlated to satisfy a basic assumption of this model.

In [30]:
#Train the BG/NBD model
bgf = BetaGeoFitter(penalizer_coef=0.1)
bgf.fit(rfm_train_test['frequency_cal'], rfm_train_test['recency_cal'], rfm_train_test['T_cal'])

#Train Gamma-Gamma
print(rfm_train_test[['monetary_value_cal', 'frequency_cal']].corr())

                    monetary_value_cal  frequency_cal
monetary_value_cal             1.00000        0.15412
frequency_cal                  0.15412        1.00000


##### We see that there is a low correlation, so we can continue.

In [31]:
ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(rfm_train_test['frequency_cal'],
        rfm_train_test['monetary_value_cal'])

<lifetimes.GammaGammaFitter: fitted with 1965 subjects, p: 1.30, q: 3.69, v: 693.76>

##### **Prediction**
##### We want to:
##### &emsp;1. Use the BG/NBD model to predict the expected number of transactions.
##### &emsp;2. Use the Gamma-Gamma model to predict the average order value.
##### &emsp;3. Multiply the average order value by the expected number of transactions to get the customer's revenue.

In [34]:
#Predict the expected number of transactions in the next 89 days
predicted_bgf = bgf.predict(89,
                        rfm_train_test['frequency_cal'], 
                        rfm_train_test['recency_cal'], 
                        rfm_train_test['T_cal'])
trans_pred = predicted_bgf.fillna(0)

#Predict the average order value
monetary_pred = ggf.conditional_expected_average_profit(rfm_train_test['frequency_cal'],
                                        rfm_train_test['monetary_value_cal'])

#Putting it all together
sales_preds = trans_pred * monetary_pred

##### **Model Evaluation**

In [35]:
actual = rfm_train_test['monetary_value_holdout'] *  rfm_train_test['frequency_holdout']

In [48]:
def evaluate_model(actual, preds, bins):
    print(f"Total Sales Actual: {np.round(actual.sum())}")
    print(f"Total Sales Predicted: {np.round(preds.sum())}")
    print(f'Mean absolute error: {mean_absolute_error(actual, preds)}')
    #merged = pd.merge(pd.DataFrame(bgf_preds), pd.DataFrame(rfm_train_test['frequency_holdout']), on=['id'])
    fig = px.scatter(x=np.array(preds), y=np.array(actual), title='Predicted vs Actual')
    fig.update_layout(
        xaxis_title='Predicted',
        yaxis_title='Actual'
    )
    fig.show()

In [49]:
evaluate_model(actual, sales_preds, bins=10)

Total Sales Actual: 1743233.0
Total Sales Predicted: 1396632.0
Mean absolute error: 556.7287963875664


# **Talk about comparison with LightGBM**

In [ ]:
#### **ML Approach**